import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 加入这行代码可以只使用cpu，对于自定义LSTM参数类训练，只使用cpu训练速度更快

In [1]:
import pandas as pd
import jieba
import re
import numpy as np
import gensim
from keras_self_attention import SeqSelfAttention
import pickle
from keras import regularizers

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
  local_device_protos = device_lib.list_local_devices()
  return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

['/device:GPU:0']


In [3]:
maxlen=100 #  在100个词后截断评论
max_words=50000 # 考虑前10000个出现的词 发现有134658个word_index，故调整为50000
embedding_dim=200

In [4]:
data=np.load("./data_50000_100.npy")
labels=np.load("./labels.npy")
Embedding_matrix=np.load("./Embedding/Embedding_matrix_200.npy")

In [5]:
data.max()

49999

In [6]:
Embedding_matrix.shape

(50000, 200)

In [7]:
from keras import layers
from keras import models
def Build_model(): # 第二层变为128
    model=models.Sequential()
    model.add(layers.Embedding(Embedding_matrix.shape[0],Embedding_matrix.shape[1],input_length=maxlen))
    # model.add(SeqSelfAttention())
    model.add(layers.Bidirectional(layers.LSTM(128,return_sequences=True,dropout=0.3)))
    # model.add(layers.LSTM(128,return_sequences=True,kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.LSTM(64,kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.layers[0].set_weights([Embedding_matrix])
    model.layers[0].trainable=False
    model.summary()
    return model

In [8]:
import keras
model=Build_model()
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['acc'])
history=model.fit(data,
          labels,
          epochs=20,
          batch_size=64,
          validation_split=0.2,
         use_multiprocessing=True )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 200)          10000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 256)          336896    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 10,423,297
Trainable params: 423,297
Non-trainable params: 10,000,000
______________________________________

In [9]:
model.save("./72.58%_200_model.h5")